In [1]:
import win32com.client
import win32com
import os
import sys
from inspect import getmembers
import pandas as pd
import dateutil.parser
from outlook import *
import xlsxwriter

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
accounts= win32com.client.Dispatch("Outlook.Application").Session.Accounts

In [3]:
f = outlook_get_folder_from_name(outlook, 'МаниМен платежи')

In [5]:
date_start=pd.to_datetime('2019-04-01', format='%Y-%m-%d')
date_end=pd.to_datetime('2019-05-15', format='%Y-%m-%d')

msg_list = extract_msg_by_dates(outlook, 'МаниМен платежи', date_start, date_end)

whole interval
2018-10-02 09:35:39 2019-05-13 13:08:52 

desired interval
2019-04-01 00:00:00 2019-05-13 13:08:52
4508 5571 

search start interval
2019-04-25 01:50:31 2019-05-13 13:08:52
4508 5571 

correcting date_end
trying index 5571, correction 0.0%, got 2019-05-13 13:08:52
correcting date_start
trying index 3907, correction -10.8%, got 2019-04-12 12:28:29
trying index 3620, correction -5.2%, got 2019-03-29 17:21:40
trying index 3668, correction -4.3%, got 2019-03-29 17:26:36
trying index 3708, correction -3.6%, got 2019-04-05 02:00:54
trying index 3638, correction -1.3%, got 2019-03-29 17:24:05
trying index 3664, correction -0.8%, got 2019-03-29 17:26:22
trying index 3680, correction -0.5%, got 2019-03-29 21:39:44
trying index 3690, correction -0.3%, got 2019-04-01 09:29:31
search result interval
2019-03-31 10:40:44 2019-05-13 13:08:52 

Загружаю 11 из 1882
Загружаю 61 из 1882
Загружаю 111 из 1882
Загружаю 161 из 1882
Загружаю 211 из 1882
Загружаю 261 из 1882
Загружаю 311 из 1882

In [11]:
df_payments = pd.DataFrame()
start = msg_list[0][0]
end = start + len(msg_list)
errors = []

for msg in msg_list:
    if msg[0] % 50 == 0:
        print('Обрабатываю {} из {}'.format(msg[0] - start, end - start))
    try:
        df_pay = None
        dfs = pd.read_html(msg[2])
        for df in dfs:
            if df.iloc[0, 0] == 'ID клиента':
                df_pay = df
        if df_pay is not None:
            df_pay.set_axis(labels=df_pay.iloc[0], axis='columns', inplace=True)
            df_pay.drop(0, inplace=True)
            df_pay.set_axis(labels=[msg[0]], axis='index', inplace=True)
            df_pay.loc[msg[0], 'inbox_date'] = msg[1]
            df_payments = df_payments.append(df_pay, sort=False)
    except Exception as e:
        errors.append([msg[0], msg[1], e])

Обрабатываю 11 из 1882
Обрабатываю 61 из 1882
Обрабатываю 111 из 1882
Обрабатываю 161 из 1882
Обрабатываю 211 из 1882
Обрабатываю 261 из 1882
Обрабатываю 311 из 1882
Обрабатываю 361 из 1882
Обрабатываю 411 из 1882
Обрабатываю 461 из 1882
Обрабатываю 511 из 1882
Обрабатываю 561 из 1882
Обрабатываю 611 из 1882
Обрабатываю 661 из 1882
Обрабатываю 711 из 1882
Обрабатываю 761 из 1882
Обрабатываю 811 из 1882
Обрабатываю 861 из 1882
Обрабатываю 911 из 1882
Обрабатываю 961 из 1882
Обрабатываю 1011 из 1882
Обрабатываю 1061 из 1882
Обрабатываю 1111 из 1882
Обрабатываю 1161 из 1882
Обрабатываю 1211 из 1882
Обрабатываю 1261 из 1882
Обрабатываю 1311 из 1882
Обрабатываю 1361 из 1882
Обрабатываю 1411 из 1882
Обрабатываю 1461 из 1882
Обрабатываю 1511 из 1882
Обрабатываю 1561 из 1882
Обрабатываю 1611 из 1882
Обрабатываю 1661 из 1882
Обрабатываю 1711 из 1882
Обрабатываю 1761 из 1882
Обрабатываю 1811 из 1882
Обрабатываю 1861 из 1882


In [12]:
df_errors = pd.DataFrame(errors)

writer = pd.ExcelWriter('moneyman_parse.xlsx', engine='xlsxwriter')
df_errors.to_excel(writer, sheet_name='errors')
df_payments.to_excel(writer, sheet_name='payments')

writer.save()